In [1]:
import os

if not os.path.exists('./answer'):
    os.makedirs('./answer')

def write_number_to_file(file_path, character):
    try:
        # Open the file in write mode
        with open(file_path, 'a') as file:
            # Write the number to the file
            file.write(character)
        print(f"Number {character} written to {file_path} successfully.")
    except Exception as e:
        print(f"Error: {e}")

#######

frame_count = 0
frame_skip = 15

#######
import pickle

import cv2
import mediapipe as mp
import numpy as np

model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

#cap = cv2.VideoCapture("./TermProjectSample.mp4")
cap = cv2.VideoCapture("./Testset_ASL_numbers.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
print("Frame rate: ", fps, "FPS")

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

labels_dict = {1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10:'10'}
while True:

    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    
    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        prediction = model.predict([np.asarray(data_aux)])

        predicted_character = labels_dict[int(prediction[0])]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    if frame_count % frame_skip == 7:
        write_number_to_file('./answer/answers.txt', predicted_character)
        
    frame_count += 1
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(50) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


Frame rate:  30.0 FPS
Number 1 written to ./answer/answers.txt successfully.
Number 2 written to ./answer/answers.txt successfully.
Number 3 written to ./answer/answers.txt successfully.
Number 5 written to ./answer/answers.txt successfully.
Number 5 written to ./answer/answers.txt successfully.
Number 6 written to ./answer/answers.txt successfully.
Number 7 written to ./answer/answers.txt successfully.
Number 8 written to ./answer/answers.txt successfully.
Number 5 written to ./answer/answers.txt successfully.
Number 1 written to ./answer/answers.txt successfully.
Number 5 written to ./answer/answers.txt successfully.
Number 10 written to ./answer/answers.txt successfully.
Number 1 written to ./answer/answers.txt successfully.
Number 9 written to ./answer/answers.txt successfully.
Number 2 written to ./answer/answers.txt successfully.
Number 8 written to ./answer/answers.txt successfully.
Number 3 written to ./answer/answers.txt successfully.
Number 7 written to ./answer/answers.txt s

AttributeError: 'NoneType' object has no attribute 'shape'